# Description of problem statement:
* After mapping the lables as proposed in problem paper, the 50% of labels were the lables number 15 and 14. Not normalized distribution of labels caused the skewing of training procedure. Namely, the model had accuracy 50% and didn't improve during the training.
* In this notebook the number of samples for each label is truncated if its exceeds the given threshold. To make the distribution normalized following thresholds were choosen: validation split - 500, test split - 500, train split - 10000.
* Train split is large and it was not possible to truncate the labels in one run because of memory consumption. That is why it was partitioned in chunks, based on the label number. After the samples inside each chunk were truncated. In the end the chunks are merged together inside the other notebook "2. merge datasets.ipynb"

In [1]:
%env XLA_FLAGS=--xla_gpu_cuda_data_dir=/opt/bwhpc/common/devel/cuda/11.8

env: XLA_FLAGS=--xla_gpu_cuda_data_dir=/opt/bwhpc/common/devel/cuda/11.8


In [2]:
import tensorflow_datasets as tfds
import os
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import json
from collections import defaultdict
from skimage import io
from skimage.color import rgb2gray
#from skimage.color import rgb2gray, rgb2grey
from scipy.ndimage.filters import gaussian_filter
import imageio
import math

2023-01-09 15:08:46.396524: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-09 15:08:51.207541: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-09 15:09:05.401251: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/bwhpc/common/devel/cuda/11.8/lib64
2023-01-09 15:09:05.415081: W tensorflow/compil

# create imagenet 2012

In [3]:
dataset_dir = '../../../dataset_files'  # directory where you downloaded the tar files to
temp_dir = '../../temp_dataset_data'

In [4]:
download_config = tfds.download.DownloadConfig(
    extract_dir=os.path.join(temp_dir, 'extracted'),
    manual_dir=dataset_dir
)

tfds.builder("imagenet2012").download_and_prepare(download_config=download_config)

# load dataset
problem: test dataset has all labeles equal to -1

In [5]:
tf.debugging.set_log_device_placement(False)
gpus = tf.config.list_logical_devices('GPU')
strategy = tf.distribute.MirroredStrategy(gpus)
with strategy.scope():
    builder = tfds.builder("imagenet2012")
    raw_valid_ds, raw_test_ds, raw_train_ds = builder.as_dataset(['validation', 'train[0:50000]', 'train[50000:]'],as_supervised=True)

2023-01-09 15:09:46.014979: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-09 15:09:48.272205: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30952 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:15:00.0, compute capability: 7.0


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [6]:
print(len(raw_train_ds))
print(len(raw_test_ds))
print(len(raw_valid_ds))

1231167
50000
50000


In [7]:

#builder = tfds.builder("imagenet2012")
#raw_train_ds, raw_valid_ds, raw_test_ds = builder.as_dataset(['train' , 'validation[50%:]','validation[:50%]'],as_supervised=True)

# prepare dataset

In [8]:
with open('./mapping.json') as json_file:
    subclass_to_class_int = json.load(json_file)
    
labels = builder.info.features['label'].names

tensor_keys = list(range(len(labels)))
tensor_values = list()
for label in labels:
    if label in subclass_to_class_int.keys():
        tensor_values.append(subclass_to_class_int[label])
    else:
        tensor_values.append(16)
        
table = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(
        keys=tf.constant(tensor_keys, dtype=tf.int64),
        values=tf.constant(tensor_values,  dtype=tf.int64),
    ),
    default_value= tf.constant(0,  dtype=tf.int64)
)

def relabel_map(x, y):
    return x, table.lookup(y)

In [9]:
num_of_lables = defaultdict(int)
c = 1

In [10]:
def resize(x, y):
    x = tf.cast(x, tf.float32) / 255.0
    x = tf.image.resize(x, (224, 224))
    return x, y


def filter_ds(val, label, max_num_samples):
    print(label)
    global c
    global num_of_lables
    print(c)
    c += 1
    label = int(label)
    num_of_lables[str(label)] += 1
    if num_of_lables[str(label)] > max_num_samples:
        return False
    else:
        return True

def prepare_ds(ds, max_num_samples):
    dataset_mapped = ds.map(resize).map(relabel_map)
    dataset_size_after = 0
    for _, label in dataset_mapped:
        if label != 16:
            dataset_size_after += 1
    dataset_filtered = dataset_mapped.filter(lambda _, l : l != 16)
    dataset_filtered = dataset_filtered.apply(tf.data.experimental.assert_cardinality(dataset_size_after))
    dataset_prepared = dataset_filtered.shuffle(10000).batch(batch_size).prefetch(4)
    return dataset_prepared

def truncate_class_samples(ds, max_num_samples):
    features = list()
    labels = list()
    num_of_lables = defaultdict(int)
    for v, label in ds:
        if label != 16:
            num_of_lables[str(label)] += 1
            if num_of_lables[str(label)] <= max_num_samples:
                features.append(v)
                labels.append(label)
    return tf.data.Dataset.from_tensor_slices((features,labels))

def prepare_ds_without_batching(ds, max_num_samples):
    dataset_mapped = ds.map(resize).map(relabel_map)
    dataset_size_after = 0
    for _, label in dataset_mapped:
        if label != 16:
            dataset_size_after += 1
    dataset_filtered = dataset_mapped.filter(lambda _, l : l != 16)
    dataset_filtered = dataset_filtered.apply(tf.data.experimental.assert_cardinality(dataset_size_after))
    #dataset_prepared = truncate_class_samples(dataset_filtered, max_num_samples)
    dataset_prepared = dataset_filtered
    return dataset_prepared

In [11]:
def count_labels(ds, name = None):
    res = defaultdict(int)
    for images, label in ds:
        numpy_label = label.numpy()
        res[str(numpy_label)] += 1
    return res

In [ ]:
tf.debugging.set_log_device_placement(False)
gpus = tf.config.list_logical_devices('GPU')
strategy = tf.distribute.MirroredStrategy(gpus)
with strategy.scope():
    ds_without_batching = prepare_ds_without_batching(raw_train_ds, 10000)
    #ds = ds_without_batching.filter(lambda _, l : l != 14 and l!=15)
    #tf.data.Dataset.save(ds, "./saved_datasets/train/0-14_ds_without_batching")
    for i in [14,15]:
        ds = ds_without_batching.filter(lambda _, l : l == i)
        tf.data.Dataset.save(ds, "./saved_datasets/train/{0}_ds_without_batching".format(str(i)))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [12]:
tf.debugging.set_log_device_placement(False)
gpus = tf.config.list_logical_devices('GPU')
strategy = tf.distribute.MirroredStrategy(gpus)
with strategy.scope():
    for ds, ds_name, max_num_samples in zip([raw_valid_ds, raw_test_ds, raw_train_ds],['valid', 'test', 'train'], [500, 500, 10000]):
        ds_without_batching = prepare_ds_without_batching(ds, max_num_samples)
        tf.data.Dataset.save(ds_without_batching, "./saved_datasets/{0}_ds_without_batching".format(ds_name))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

2023-01-08 16:26:18.897594: I tensorflow/tsl/framework/bfc_allocator.cc:1090] InUse at 150e02e95200 of size 256 next 59604
2023-01-08 16:26:18.897597: I tensorflow/tsl/framework/bfc_allocator.cc:1090] InUse at 150e02e95300 of size 256 next 59605
2023-01-08 16:26:18.897600: I tensorflow/tsl/framework/bfc_allocator.cc:1090] InUse at 150e02e95400 of size 256 next 59606
2023-01-08 16:26:18.897603: I tensorflow/tsl/framework/bfc_allocator.cc:1090] InUse at 150e02e95500 of size 2

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run Pack: Dst tensor is not initialized. [Op:Pack] name: component_0

In [ ]:
print("ready")